In [1]:
!pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pprint
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

**Choose Artists**

---



In [3]:
# Destiny's child link
url_DC='https://www.lyrics.com/album/2571820/Playlist%3A-The-Very-Best-of-Destiny%27s-Child'

In [4]:
# Radiohead link
url_Radiohead='https://www.lyrics.com/album/1382940/The-Best-Of-%5BUS-Limited-Edition%5D'

In [5]:
# def to read URL to soup
def read_URL_to_Soup(url):
  response=requests.get(url)
  html = response.text
  html_soup = BeautifulSoup(html, 'html.parser')
  return(html_soup)

**Read HTML pages of both artists**

---





In [6]:
DC=read_URL_to_Soup(url_DC)
Radiohead=read_URL_to_Soup(url_Radiohead)

In [7]:
DC


<!DOCTYPE doctype html>

<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]--><head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Various Artists - Playlist: The Very Best of Destiny's Child Album | Lyrics.com</title>
<meta content="Playlist: The Very Best of Destiny's Child is the third compilation album by American R&amp;B girl group Destiny's Child. It was released on October 9, 2012 through Columbia Records matching with the fifteenth anniversary of Destiny's Child's formation. The compilation contained fourteen songs from the group's repertoire consisting of four studio albums. Upon its release, the received positive reviews from critics who praised its track list featuring the band's most 

In [8]:
Radiohead


<!DOCTYPE doctype html>

<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]--><head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Radiohead - The Best Of [US Limited Edition] Album | Lyrics.com</title>
<meta content="All the great songs and lyrics from the &amp;quot;The Best Of [US Limited Edition]&amp;quot; album on the Web's largest and most authoritative lyrics resource." name="description"/>
<meta content="The Best Of [US Limited Edition] album, The Best Of [US Limited Edition] songs, The Best Of [US Limited Edition] lyrics" name="keywords"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<base href="https://www.lyrics.com/"/>
<script>
s4Prefix = 'http

In [9]:
DC.head.title.text

"Various Artists - Playlist: The Very Best of Destiny's Child Album | Lyrics.com"

In [10]:
Radiohead.head.title.text

'Radiohead - The Best Of [US Limited Edition] Album | Lyrics.com'

**Get the hyperlinks to lyrics pages**

---



In [11]:
# find all url links using BeautifulSoup
def get_all_URLs(html_soup):
  urls=[]
  for tag in html_soup.findAll('a', href=True):
    urls.append(tag['href'])
  return(urls)

In [12]:
DC_urls=get_all_URLs(DC)
#DC_urls
Radiohead_urls=get_all_URLs(Radiohead)
#Radiohead_urls

In [13]:
# get all the url links to songs
def get_lyrics_URLs(urls_list):
  prefix_url ='https://www.lyrics.com'
  complete_url_DC_list = []

  for hyper_link in urls_list:
    hyper_list_DC = re.findall(pattern='/lyric/.*', string=hyper_link)
    if  not not hyper_list_DC: # if the list is not empty
        complete_url = prefix_url + hyper_list_DC[0]
        complete_url_DC_list.append(complete_url)
  return(complete_url_DC_list)      

lyrics_url_DC=get_lyrics_URLs(DC_urls)
lyrics_url_Radiohead=get_lyrics_URLs(Radiohead_urls)

**Get the lyrics in all pages**

---



In [14]:
# get the lyrics text body
def get_Lyrics(html_soup):
  html_body_lyrics_soup= html_soup.body.find(id="lyric-body-text")
  return(html_body_lyrics_soup)

# get lyrics text and clean it 
def clean_Lyrics(html_body_lyrics_soup):
  html_no_tag = re.sub('<.*?>' ,'', str(html_body_lyrics_soup))
  # remove non alphabetical characters 
  html_clean1=re.sub('\r', '', html_no_tag)
  html_clean2=re.sub('\n', ' ', html_clean1)
  html_clean3=re.sub('\W', ' ',html_clean2)
  return(html_clean3)

# all together get Lyrics
def get_clean_Lyrics(url):
   return clean_Lyrics(get_Lyrics(read_URL_to_Soup(url)))

# get Song title  
def get_song_title(html_soup):
  return(html_soup.head.title.text)


In [15]:
lyrics_url_DC

['https://www.lyrics.com/lyric/27756259/Bootylicious',
 'https://www.lyrics.com/lyric/27756258/Bug+a+Boo+%5BH-town+Screwed+Mix%5D',
 'https://www.lyrics.com/lyric/27756257/Emotion+%5BThe+Neptunes+Remix%5D',
 'https://www.lyrics.com/lyric/27756256/Jumpin%27%2C+Jumpin%27',
 'https://www.lyrics.com/lyric/32463822/Jumpin%27%2C+Jumpin%27+%5BSo+So+Def+Remix%5D',
 'https://www.lyrics.com/lyric/27756255/Independent+Women+Part+I+%5B%231%27s+Edit%5D+%5BEdit%5D',
 'https://www.lyrics.com/lyric/27756254/Say+My+Name+%5B%231%27s+Edit%5D+%5BEdit%5D',
 'https://www.lyrics.com/lyric/27756250/No%2C+No%2C+No%2C+Pt.+2',
 'https://www.lyrics.com/lyric/27756249/Survivor+%5B%231%27s+Edit%5D+%5BEdit%5D',
 'https://www.lyrics.com/lyric/27756248/Lose+My+Breath',
 'https://www.lyrics.com/lyric/27756247/So+Good',
 'https://www.lyrics.com/lyric/27756246/Girl',
 'https://www.lyrics.com/lyric/32463829/Girl+%5BVersion%5D',
 'https://www.lyrics.com/lyric/27756253/Bills%2C+Bills%2C+Bills+%5B%231%27s+Edit%5D+%5BEdit%5D'

# **First Artist**

---



In [16]:
# all lyrics together
song_title_DC=[]
corpus_DC=[]

for url in lyrics_url_DC:
  corpus_DC.append(get_clean_Lyrics(url))
  song_title_DC.append(get_song_title(read_URL_to_Soup(url)))

**Check for repeated songs in titles**

---



In [17]:
song_title_DC

["Destiny's Child - Bootylicious Lyrics | Lyrics.com",
 "Destiny's Child - Bug a Boo [H-town Screwed Mix] Lyrics | Lyrics.com",
 "Destiny's Child - Emotion [The Neptunes Remix] Lyrics | Lyrics.com",
 "Destiny's Child - Jumpin', Jumpin' Lyrics | Lyrics.com",
 "Da Brat - Jumpin', Jumpin' [So So Def Remix] Lyrics | Lyrics.com",
 "Destiny's Child - Independent Women Part I [#1's Edit] [Edit] Lyrics | Lyrics.com",
 "Destiny's Child - Say My Name [#1's Edit] [Edit] Lyrics | Lyrics.com",
 'Wyclef Jean - No, No, No, Pt. 2 Lyrics | Lyrics.com',
 "Destiny's Child - Survivor [#1's Edit] [Edit] Lyrics | Lyrics.com",
 "Destiny's Child - Lose My Breath Lyrics | Lyrics.com",
 "Destiny's Child - So Good Lyrics | Lyrics.com",
 "Destiny's Child - Girl Lyrics | Lyrics.com",
 "Destiny's Child - Girl [Version] Lyrics | Lyrics.com",
 "Destiny's Child - Bills, Bills, Bills [#1's Edit] [Edit] Lyrics | Lyrics.com",
 "Destiny's Child - Soldier [Radio Edit] [Edit] Lyrics | Lyrics.com",
 'Wyclef Jean - Illusion L

**Remove repeated song**

In [18]:
len(corpus_DC)

16

In [19]:
del corpus_DC[11] # the list number of repeated song

In [20]:
corpus_DC

['Kelly  can you handle this  Michelle  can you handle this  Beyoncé  can you handle this  I don t think they can handle this   Barely move  we ve arrived Lookin  sexy  lookin  fly Baddest chick  chick inside DJ  jam tonight Spotted me a tender thang There you are  come on baby Don t you wanna dance with me Can you handle  handle me  You gotta do much better if you gone dance with me tonight You gotta work your jelly if you gone dance with me tonight Read my lips carefully if you like what you see Move  groove  prove you can hang with me By the looks I got you shook up and scared of me Hook up your seat belt  it s time for takeoff  I don t think you ready for this jelly I don t think you ready for this jelly I don t think you ready for this  Cause my body too bootylicious for yo babe  I don t think you ready for this jelly I don t think you ready for this jelly I don t think you ready for this  Cause my body too bootylicious for yo babe  Baby  can you handle this  Baby  can you handle 

In [21]:
len(corpus_DC)

15

# **Second Artist**

---



In [22]:
# all lyrics together
song_title_Radiohead=[]
corpus_Radiohead=[]

for url in lyrics_url_Radiohead:
  corpus_Radiohead.append(get_clean_Lyrics(url))
  song_title_Radiohead.append(get_song_title(read_URL_to_Soup(url)))

In [23]:
song_title_Radiohead

['Radiohead - Just Lyrics | Lyrics.com',
 'Radiohead - Paranoid Android Lyrics | Lyrics.com',
 'Radiohead - Karma Police Lyrics | Lyrics.com',
 'Radiohead - Creep Lyrics | Lyrics.com',
 'Radiohead - No Surprises Lyrics | Lyrics.com',
 'Radiohead - High and Dry Lyrics | Lyrics.com',
 'Radiohead - My Iron Lung Lyrics | Lyrics.com',
 'Radiohead - There There Lyrics | Lyrics.com',
 'Radiohead - Lucky Lyrics | Lyrics.com',
 'Radiohead - Optimistic Lyrics | Lyrics.com',
 'Radiohead - Fake Plastic Trees Lyrics | Lyrics.com',
 'Radiohead - Idioteque Lyrics | Lyrics.com',
 'Radiohead - 2+2=5 Lyrics | Lyrics.com',
 'Radiohead - The Bends Lyrics | Lyrics.com',
 'Radiohead - Pyramid Song Lyrics | Lyrics.com',
 'Radiohead - Street Spirit (Fade Out) Lyrics | Lyrics.com',
 'Radiohead - Everything in Its Right Place Lyrics | Lyrics.com',
 'Radiohead - Airbag [*] Lyrics | Lyrics.com',
 'Radiohead - I Might Be Wrong [*] Lyrics | Lyrics.com',
 'Radiohead - Go to Sleep [*] Lyrics | Lyrics.com',
 'Radiohea

In [24]:
corpus_Radiohead

['Can t get the stink off He s been hanging around for days Comes like a comet Suckered you but not your friends  One day he ll get to you And teach you how to be a holy cow  You do it to yourself  you do And that s what really hurts Is you do it to yourself  just you You and no one else You do it to yourself You do it to yourself  Don t get my sympathy  hanging out the fifteenth floor You ve changed the locks three times He still comes reeling through the door One day I ll get to you And teach you how to get to purest hell  You do it to yourself you do And that s what really hurts Is you do it to yourself  just you You and no one else You do it to yourself You do it to yourself  You do it to yourself you do And that s what really hurts Is you do it to yourself just you You and no one else You do it to yourself You do it to yourself  whoa  Say  hey  You do it to yourself  you do it to yourself Yeah  yeah  you do it to yourself  yes yes  you Do it to yourself',
 'Please could you stop t

In [25]:
len(corpus_Radiohead)

30

**Preprocessing and feature engineering for bag of words**

---



---



In [26]:
# Concatenate the lists
corpus=corpus_DC + corpus_Radiohead
#corpus

In [27]:
labels = ['Destinys Child'] * 15 + ['Radiohead'] * 30

In [51]:
print(labels)

['Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Destinys Child', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead', 'Radiohead']


In [50]:
# convert labels to 0 and 1
def get_dummy(label_list, positive_label):
  label_list_binary=[]
  for label in label_list:
    if label == positive_label:
      label_list_binary.append(1)
    else:
      label_list_binary.append(0)
  return(label_list_binary)

labels_binary=get_dummy(labels,'Radiohead')
print(labels_binary)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


**Split train test**

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(corpus, labels_binary, random_state=10, stratify=labels_binary)

**Pipeline feature engineering and model training**

In [63]:
# Pipeline to feature engineer and train naive bayes model
pipeline = Pipeline( [('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', MultinomialNB())] )

In [64]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [65]:
# print training set accuracy
train_acc = round(pipeline.score(X_train, y_train) * 100, 2)
print("Prediction accuracy (Training Set):", train_acc, "%\n")

# Check and print prediction accuracy and model parameters
test_acc=round(pipeline.score(X_test, y_test) * 100,2)
print("Prediction accuracy (Test Set):", test_acc, "%\n")

Prediction accuracy (Training Set): 72.73 %

Prediction accuracy (Test Set): 66.67 %

